In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import sys
import logging

from neoglia.workers.connect_workers import connect
from neoglia.learn.utils import setup_logging
from neoglia.learn.config import LearnConfig
from neoglia.learn.losses import binary_cross_entropy
from neoglia.learn.models import FFNet
from neoglia.learn.learner import Learner

In [25]:
# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create STDERR handler
handler = logging.StreamHandler(sys.stderr)
# ch.setLevel(logging.DEBUG)

# Create formatter and add it to the handler
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Set STDERR handler as the only handler 
logger.handlers = [handler]

## Connect to data nodes

In this demo, we have 3 distinct hospitals. Each is an indenpendent EC2 instance on AWS.

In [26]:
h1, h2, h3 = connect(local=True)

root - WARNING - Torch was already hooked... skipping hooking process
neoglia.workers.connect_workers - INFO - Connected to worker h1.
neoglia.workers.connect_workers - INFO - Connected to worker h2.
neoglia.workers.connect_workers - INFO - Connected to worker h3.


Check the datasets they have and the dimensions of these.

In [27]:
logger.info(h1.list_datasets())

root - INFO - -mnist_train:
	data size: [24754, 28, 28],
	target size: [24754]
-mnist_test:
	data size: [10000, 28, 28],
	target size: [10000]
-eicu_class_train:
	data size: [4778, 103],
	target size: [4778]
-eicu_class_test:
	data size: [5421, 103],
	target size: [5421]
-eicu_reg_train:
	data size: [4778, 103],
	target size: [4778]
-eicu_reg_test:
	data size: [5421, 103],
	target size: [5421]



## Train a convolutional neural network on the mnist dataset with federated averaging

Each hospital holds a subset of the training data but they all share the same test data.

## Define the config file for this experiment

This holds everything from the learning rate to the batch size. 

First let's check the available parameters. Note, this object can take a yml config file (good for reproducible experiments) or be parametrised when instantiated.

In [10]:
?LearnConfig

Init signature:
LearnConfig(
    ['config_file=None', 'train_dataset_name=None', 'test_dataset_name=None', 'train_batch_size=64', 'test_batch_size=128', 'train_epochs=40', 'fed_after_n_batches=10', 'lr=0.1', 'cuda=False', 'seed=42', 'save_model=True', 'verbose=True'],
)
Docstring:      Config dict object, holding all parameters for the training and evaluation.
Init docstring:
Constructor of the subclassed dict object.

Args:
    config_file (str): Location of config YAML file. If provided, all
        parameters that are defined within will override the defaults here.
    train_dataset_name (str): Name of the remote dataset to train on.
    test_dataset_name (str): Name of the remote dataset to test on.
    train_batch_size (int): Batch size for training.
    test_batch_size (int): Batch size for evaluation.
    train_epochs (int): Number of epochs performed altogether for training on
        remote workers.
    fed_after_n_batches (int): Number of training epochs performed on each
   

In [11]:
config = LearnConfig("eicu_class_config.yml")
config.fed_after_n_batches = 5
config.train_epochs = 10
config

{'config_file': 'eicu_class_config.yml',
 'train_dataset_name': 'eicu_class_train',
 'test_dataset_name': 'eicu_class_test',
 'train_batch_size': 64,
 'test_batch_size': 128,
 'train_epochs': 10,
 'fed_after_n_batches': 5,
 'lr': 0.1,
 'cuda': False,
 'seed': 42,
 'save_model': True,
 'verbose': True}

## Define model architecture and loss function

Define a model architecture in Torch, or simply load one of NeoGlia's predefined ones.

In [22]:
model = FFNet(input_d=103, final='sigmoid')

In [18]:
%psource FFNet

class FFNet(nn.Module):
    """
    Feed-forward network with dropout on tabular, numeric data.

    Returns probabilities after sigmoid and not logits.
    """
    def __init__(self, input_d, dense_size=256, dropout_rate=0.25, final=None):
        """
        Constructor of neural net.

        Args:
            input_d (int): Input dimensions to the first layers, i.e. num cols in data.
            dense_size (int): Size of the first layer. There are 3 layers in this
                architecture, the 2nd has dense_size/2 the 3rd has dense_size/4 units.
            dropout_rate (float): Rate of dropout applied to each fully connected layers.
            final (str): What should the final activation layer be. Options are:
                sigmoid, relu, None.
        """
        super(FFNet, self).__init__()
        self.fc1 = nn.Linear(input_d, dense_size)
        self.fc2 = nn.Linear(dense_size, int(dense_size/2))
        self.fc3 = nn.Linear(int(dense_size/2), int(dense_size/4))
     

We'll use cross entropy in this example as a loss function as this is a multi-class problem.

## Start training and evaluating the model in a federated manner. 

In [23]:
model()

FFNet(
  (fc1): Linear(in_features=103, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
  (do): Dropout(p=0.25)
  (final_act): Sigmoid()
)

In [19]:
fed_learner = Learner(
    config=config,
    model=model, 
    model_input_dim=[1, 103],
    loss_fn=binary_cross_entropy, 
    workers=(h1, h2, h3)
)

In [20]:
fed_learner.train_eval()

neoglia.learn.learner - INFO - Starting epoch 1/10
neoglia.learn.learner - INFO - Training round: 1, worker: h3, avg_loss: 0.6045
neoglia.learn.learner - INFO - Training round: 1, worker: h1, avg_loss: 0.5584
neoglia.learn.learner - INFO - Training round: 1, worker: h2, avg_loss: 0.5278
neoglia.learn.learner - INFO - Starting epoch 2/10
neoglia.learn.learner - INFO - Training round: 2, worker: h2, avg_loss: 0.5115
neoglia.learn.learner - INFO - Training round: 2, worker: h1, avg_loss: 0.5891
neoglia.learn.learner - INFO - Training round: 2, worker: h3, avg_loss: 0.4935
neoglia.learn.learner - INFO - Starting epoch 3/10
neoglia.learn.learner - INFO - Training round: 3, worker: h2, avg_loss: 0.3881
neoglia.learn.learner - INFO - Training round: 3, worker: h3, avg_loss: 0.4747
neoglia.learn.learner - INFO - Training round: 3, worker: h1, avg_loss: 0.4219
neoglia.learn.learner - INFO - Starting epoch 4/10
neoglia.learn.learner - INFO - Training round: 4, worker: h3, avg_loss: 0.3763
neogli

RuntimeError: Websocket connection closed and creation of new connection failed.